In [1]:
from pathlib import Path

import gdsfactory as gf
from gdsfactory import Component
from gdsfactory.typings import LayerSpec

from typing import Literal, Any, Union, TypeAlias, List
import numpy as np
from numpy import floor, linspace, round, zeros, ndarray

from ihp import tech, cells, PDK

In [2]:
PDK.activate()

In [ ]:
FloatLike: TypeAlias = Union[np.float32, np.float64, float]
Point: TypeAlias = tuple[FloatLike, FloatLike]

def guard_ring(
    width: float = 0.5,
    guardRingSpacing: float = 0.14,
    guardRingType: Literal["psub", 'nwell'] = "pwell",
    bbox: Union[tuple[Point, Point], ndarray[Point], None] = None,
    path: Union[List[tuple[Point, Point]], ndarray[Point], None] = None,
    layer_activ: LayerSpec = "Activdrawing",
    layer_cont: LayerSpec = "Contdrawing",
    layer_metal1: LayerSpec = "Metal1drawing",
    layer_psd: LayerSpec = "pSDdrawing",
    layer_nwell: LayerSpec = "NWelldrawing",
    layer_nsd: LayerSpec = "nSDdrawing",
    **kwargs
) -> Component:
    """
    Create a closed np or pplus guard ring around a boundary box, 
    or, if bbox is not provided, along a path of points.
    """
    
    gr_drc = {
        # metals
        'm1_min_width': tech.TECH.metal1_width,
        # active regions and contact
        'cont_min_size': tech.TECH.cont_size,
        'cont_min_spacing': tech.TECH.cont_spacing,
        'cont_min_enclose_active': tech.TECH.cont_enc_active,
        'cont_min_enclose_metal': tech.TECH.cont_enc_metal,
        # TODO: add in the original tech struct
        'active_min_enclose_np': 0.14,
        'active_min_enclose_pp': 0.14,
        'np_min_enclose_nw': 0.14,
    }

    min_width = gr_drc['cont_min_size'] + \
        2*max(gr_drc['cont_min_enclose_active'], gr_drc['cont_min_enclose_metal'])
    min_width = max(min_width, gr_drc['m1_min_width'])
    assert width >= min_width, \
        f"Guard Ring width >= {min_width} to comply with Min cont enclosure and metal width"

    # define nrows and ncols of the required tap
    nrows = int(floor(width / min_width))
    c = Component()
    
    # define the path 
    if bbox is not None:
        
        path = [
            (bbox[0][0] - guardRingSpacing - width/2, bbox[1][1] + guardRingSpacing + width/2),
            (bbox[1][0] + guardRingSpacing + width/2, bbox[1][1] + guardRingSpacing + width/2),
            (bbox[1][0] + guardRingSpacing + width/2, bbox[0][1] - guardRingSpacing - width/2),
            (bbox[0][0] - guardRingSpacing - width/2, bbox[0][1] - guardRingSpacing - width/2),
            (bbox[0][0] - guardRingSpacing - width/2, bbox[1][1] + guardRingSpacing + width),
        ]
        enclosure = max(gr_drc['cont_min_enclose_active'], gr_drc['cont_min_enclose_metal'])
        cont_path = [
            (bbox[0][0] - guardRingSpacing - enclosure, bbox[1][1] + guardRingSpacing + enclosure),
            (bbox[1][0] + guardRingSpacing + enclosure, bbox[1][1] + guardRingSpacing + enclosure),
            (bbox[1][0] + guardRingSpacing + enclosure, bbox[0][1] - guardRingSpacing - enclosure),
            (bbox[0][0] - guardRingSpacing - enclosure, bbox[0][1] - guardRingSpacing - enclosure),
            (bbox[0][0] - guardRingSpacing - enclosure, bbox[1][1] + guardRingSpacing ),
        ]

    assert path is not None, "Neither path or bbox was provided."
    # place taps around path
    tap_layers = [layer_activ, layer_metal1]
    main = None
    for layer_spec in tap_layers:
        p = gf.path.extrude(gf.path.Path(path), width = width, layer=layer_spec)
        main = c.add_ref(p)
    if guardRingType == 'psub':
        sep = gr_drc['active_min_enclose_pp']
        last_point = list(path[-1])
        last_edge = (path[-1][0] -path[-2][0], path[-1][1]-path[-2][1])
        norm = np.linalg.norm(last_edge)
        dir_vec = np.array(last_edge) / norm
        # manhattan
        dir_vec[0] = round(dir_vec[0])
        dir_vec[1] = round(dir_vec[1]) 
        last_point[0] += sep*dir_vec[0]
        last_point[1] += sep*dir_vec[1]
        new_path = path.copy()
        new_path[-1] = tuple(last_point)
        p = gf.path.extrude(gf.path.Path(new_path), width = width+2*sep, layer=layer_psd)
        c.add_ref(p)
    if guardRingType == 'nwell':
        sep = gr_drc['active_min_enclose_np']
        last_point = list(path[-1])
        last_edge = (path[-1][0] -path[-2][0], path[-1][1]-path[-2][1])
        norm = np.linalg.norm(last_edge)
        dir_vec = np.array(last_edge) / norm
        # manhattan
        dir_vec[0] = round(dir_vec[0])
        dir_vec[1] = round(dir_vec[1]) 
        last_point[0] += sep*dir_vec[0]
        last_point[1] += sep*dir_vec[1]
        new_path = path.copy()
        new_path[-1] = tuple(last_point)
        p = gf.path.extrude(gf.path.Path(new_path), width = width+2*sep, layer=layer_nsd)
        
        sep += gr_drc['np_min_enclose_nw']
        last_point = list(path[-1])
        last_point[0] += sep*dir_vec[0]
        last_point[1] += sep*dir_vec[1]
        new_path = path.copy()
        new_path[-1] = tuple(last_point)
        nwl = gf.path.extrude(gf.path.Path(new_path), width = width+2*sep, layer=layer_nwell)
        c.add_ref(p)
        c.add_ref(nwl)

    cont_tap = cells.via_array(
        via_type = layer_cont.split('drawing')[0],
        via_size = gr_drc['cont_min_size'],
        via_spacing = gr_drc['cont_min_size'] + gr_drc['cont_min_spacing'],
        via_enclosure = gr_drc['cont_min_enclose_active'],
        columns = 1,
        rows = nrows
    )

    conts = gf.path.along_path(
        gf.path.Path(cont_path if bbox is not None else path), 
        cont_tap, gr_drc['cont_min_spacing'] + gr_drc['cont_min_size'],0.0)
    cont_ref = c.add_ref(conts)
    cont_ref.x = main.x 
    cont_ref.y = main.y
    c.info['model'] = f'{guardRingType}-guard-ring'
    c.info['width'] = width
    c.info['rows'] = nrows
    c.info['guardRingSpacing'] = guardRingSpacing
    
    return c

In [ ]:
GDS_PATH = Path('../gds').resolve()
GDS_PATH

PosixPath('/mnt/c/Users/dasdi/Documents/GDSFactory/diogo/cells/design_data/gds')

In [18]:
c = Component()

c.add_ref(gf.components.rectangle(
    (10,10), layer="Metal1drawing"))

pgr = guard_ring(0.4, 0.3, bbox=c.bbox_np(), guardRingType="psub")
c.add_ref(pgr)

ngr = guard_ring(1.0, 0.6, bbox= c.bbox_np(), guardRingType="nwell")
c.add_ref(ngr)


line_gr = guard_ring(1.0, 0.3, path = [(10.0, 20.0), (12.0, 20.0), (12.0, 30.0)], guardRingType="psub")
c.add_ref(line_gr)

Unnamed_47: ports [], KCell(name=Unnamed_59, ports=[], pins=[], instances=['Unnamed_60_0_0', 'Unnamed_61_0_0', 'Unnamed_62_0_0', 'Unnamed_63_225_-305'], locked=False, kcl=DEFAULT)

In [19]:
c.write_gds(GDS_PATH / "guard_ring_example.gds")

PosixPath('/mnt/c/Users/dasdi/Documents/GDSFactory/diogo/cells/design_data/gds/guard_ring_example.gds')

In [89]:
def cmim(
    width: float = 6.0,
    length: float = 6.0,
    bot_enclosure: float = 0.6,
    top_enclosure: float = 0.36,
    layer_metal5: LayerSpec = "Metal5drawing",
    layer_mim: LayerSpec = "MIMdrawing",
    layer_vmim: LayerSpec = "Vmimdrawing",
    layer_topmetal1: LayerSpec = "TopMetal1drawing",
    layer_cap_mark: LayerSpec = "MemCapdrawing",
    layer_m4nofill: LayerSpec = "Metal4nofill",
    layer_m5nofill: LayerSpec = "Metal5nofill",
    layer_tm1nofill: LayerSpec = "TopMetal1nofill",
    layer_tm2nofill: LayerSpec = "TopMetal2nofill",
    layer_text: LayerSpec = "TEXTdrawing",
    layer_metal5label: LayerSpec = "Metal5label",
    layer_topmetal1label: LayerSpec = "TopMetal1label",
    layer_metal5pin: LayerSpec = "Metal5pin",
    layer_topmetal1pin: LayerSpec = "TopMetal1pin",
    model: str = 'cmim'
) -> Component:
    """Create a MIM (Metal-Insulator-Metal) capacitor.

    Args:
        width: Width of the capacitor in micrometers.
        length: Length of the capacitor in micrometers.
        capacitance: Target capacitance in fF (optional).
        model: Device model name.
        layer_metal4: Bottom plate metal layer.
        layer_metal5: Top plate metal layer.
        layer_mim: MIM dielectric layer.
        layer_via4: Via layer for top plate connection.
        layer_topmetal1: Top metal layer for connections.
        layer_topvia1: Via to top metal layer.
        layer_cap_mark: Capacitor marker layer.
        layer_nofill: No metal filler layer.

    Returns:
        Component with MIM capacitor layout.
    """

    c = Component()

    mim_drc = {
        # capacitor
        'mim_min_size': tech.TECH.mim_min_size,
        'mim_cap_density': tech.TECH.mim_cap_density,
        # metals
        'm5_min_width': tech.TECH.metal5_width,
        'm5_min_spacing': tech.TECH.metal5_spacing,
        'topmetal1_width': tech.TECH.topmetal1_width,
        'topmetal1_spacing': tech.TECH.topmetal1_spacing,
        # vias
        'vmim_size': 0.42,
        'vmim_spacing': 0.52,
        'vmim_enc_metal': 0.42
    }
    
    # verification
    assert width > mim_drc['mim_min_size'], f"MIM width > {mim_drc['mim_min_size']}"
    assert length > mim_drc['mim_min_size'], f"MIM width > {mim_drc['mim_min_size']}"
    
    # snap to grid
    grid = tech.TECH.grid
    width = round(width / grid) * grid
    length = round(length / grid) * grid

    # build capacitor stack
    # Bottom plate (Metal4)
    bottom_plate_width = width + 2 * bot_enclosure + 2*top_enclosure
    bottom_plate_length = length + 2 * bot_enclosure + 2*top_enclosure

    bottom_plate = gf.components.rectangle(
        size=(bottom_plate_length, bottom_plate_width),
        layer=layer_metal5,
        centered=True,
    )
    bot = c.add_ref(bottom_plate)

    # MIM layer
    mim_layer = gf.components.rectangle(
        size=(length + 2*top_enclosure, width + 2*top_enclosure),
        layer=layer_mim,
        centered=True,
    )
    c.add_ref(mim_layer)

    # add vmim via array
    vmim_min_width = mim_drc['vmim_size'] + mim_drc['vmim_spacing']
    nrows = int(floor(width / vmim_min_width))
    ncols = int(floor(length / vmim_min_width))

    # Top plate (TopMetal1)
    top_plate = gf.components.rectangle(
        size=(length, width),
        layer=layer_topmetal1,
        centered=True,
    )
    top = c.add_ref(top_plate)

    vmim_array = cells.via_array(
        via_type = layer_vmim.split('drawing')[0],
        via_size = mim_drc['vmim_size'],
        via_spacing = mim_drc['vmim_size'] + mim_drc['vmim_spacing'],
        via_enclosure = mim_drc['vmim_enc_metal'],
        columns = ncols,
        rows = nrows
    )
    vias = c.add_ref(vmim_array)
    vias.x = top.x
    vias.y = top.y


    logic = [
        layer_cap_mark, layer_m4nofill, 
        layer_m5nofill, layer_tm1nofill, 
        layer_tm2nofill
    ]
    for layer_spec in logic:
        ll = gf.components.rectangle(
            size=(bottom_plate_length, bottom_plate_width),
            layer=layer_spec,
            centered=True,
        )
        c.add_ref(ll)

    minus = c.add_port(
        name="MINUS",
        center=(bot.xmin + mim_drc['m5_min_width']/2, bot.y),
        width=mim_drc['m5_min_width'],
        orientation=180,
        layer=layer_metal5label,
        port_type="electrical",
    )

    plus = c.add_port(
        name="PLUS",
        center=(top.xmax - mim_drc['topmetal1_width']/2, top.y),
        width=mim_drc['topmetal1_width'],
        orientation=0,
        layer=layer_topmetal1label,
        port_type="electrical",
    )

    pin_minus = gf.components.rectangle(
        size=(mim_drc['topmetal1_width'], 2*mim_drc['topmetal1_width']),
        layer=layer_metal5pin,
        centered=True,
    )
    pin_minus_ref = c.add_ref(pin_minus)
    pin_minus_ref.xmin = bot.xmin
    pin_minus_ref.y = minus.y

    pin_plus = gf.components.rectangle(
        size=(mim_drc['topmetal1_width'], 2*mim_drc['topmetal1_width']),
        layer=layer_topmetal1pin,
        centered=True,
    )
    pin_plus_ref = c.add_ref(pin_plus)
    pin_plus_ref.xmax = top.xmax
    pin_plus_ref.y = plus.y

    c.add_label(
        text="PLUS",
        position=(top.xmax - mim_drc['topmetal1_width']/2, top.y),
        layer=layer_text
    )
    c.add_label(
        text="MINUS",
        position=(bot.xmin + mim_drc['m5_min_width']/2, bot.y),
        layer=layer_text
    )

    c.add_label(
        text=model,
        position=(c.x, c.y + width/2),
        layer=layer_text
    )

    capacitance = width * length * mim_drc['mim_cap_density']
    
    c.add_label(
        text=f"C = {capacitance} fF",
        position=(c.x, c.y - width/2),
        layer=layer_text
    )

    c.info["model"] = model
    c.info["width"] = width
    c.info["length"] = length
    c.info["capacitance_fF"] = capacitance
    c.info["area_um2"] = width * length
    return c

In [90]:
c = Component()
cap = cmim()
c.add_ref(cap)

Unnamed_232: ports ['MINUS', 'PLUS'], KCell(name=Unnamed_233, ports=['MINUS', 'PLUS'], pins=[], instances=['rectangle_gdsfactorypcomponentspshapesprectangle_S7p92__2f53f460_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S6p72__84c52099_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S6_6_L_617c6e3a_0_0', 'via_array_VTVmim_C6_R6_VS0p42_VS0p94_VE0p42_LCContdrawi_f9908038_-2560_-2560', 'rectangle_gdsfactorypcomponentspshapesprectangle_S7p92__94a7ff8e_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S7p92__140be9b3_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S7p92__4ea18f77_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S7p92__2ec7e15c_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S7p92__59f34193_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S1_2_L_313130ad_-3460_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S1_2_L_21edbbcb_2500_0'], locked=False, kcl=DEFAULT)

In [91]:
c.write_gds(GDS_PATH / "cmim_example.gds")

PosixPath('/mnt/c/Users/dasdi/Documents/GDSFactory/diogo/cells/design_data/gds/cmim_example.gds')

In [98]:
def rfcmim(
    width: float = 6.0,
    length: float = 6.0,
    layer_pwellblock: LayerSpec = "PWellblock",
    layer_metal5: LayerSpec = "Metal5drawing",
    layer_mim: LayerSpec = "MIMdrawing",
    layer_vmim: LayerSpec = "Vmimdrawing",
    layer_topmetal1: LayerSpec = "TopMetal1drawing",
    layer_cap_mark: LayerSpec = "MemCapdrawing",
    layer_m4nofill: LayerSpec = "Metal4nofill",
    layer_m5nofill: LayerSpec = "Metal5nofill",
    layer_tm1nofill: LayerSpec = "TopMetal1nofill",
    layer_tm2nofill: LayerSpec = "TopMetal2nofill",
    layer_activ: LayerSpec = "Activdrawing",
    layer_cont: LayerSpec = "Contdrawing",
    layer_metal1: LayerSpec = "Metal1drawing",
    layer_psd: LayerSpec = "pSDdrawing",
    layer_rfpad: LayerSpec = "RFPaddrawing",
    layer_activnoqrc: LayerSpec = "Activnoqrc",
    layer_metal1noqrc: LayerSpec = "Metal1noqrc",
    layer_metal2noqrc: LayerSpec = "Metal2noqrc",
    layer_metal3noqrc: LayerSpec = "Metal3noqrc",
    layer_metal4noqrc: LayerSpec = "Metal4noqrc",
    layer_metal5noqrc: LayerSpec = "Metal5noqrc",
    layer_topmetal1noqrc: LayerSpec = "TopMetal1noqrc",
    layer_text: LayerSpec = "TEXTdrawing",
    layer_metal1pin: LayerSpec = "Metal1pin",
    layer_metal5pin: LayerSpec = "Metal5pin",
    layer_topmetal1pin: LayerSpec = "TopMetal1pin",
    layer_metal5label: LayerSpec = "Metal5label",
    layer_topmetal1label: LayerSpec = "TopMetal1label",
    layer_metal1label: LayerSpec = "Metal1label",
    model: str = 'rfcmim'
) -> Component:

    c = Component()
    
    cap = cmim(
        width=width,
        length=length,
        bot_enclosure=0.6,
        top_enclosure=0.36,
        layer_metal5=layer_metal5,
        layer_mim=layer_mim,
        layer_vmim=layer_vmim,
        layer_topmetal1=layer_topmetal1,
        layer_cap_mark=layer_cap_mark,
        layer_m4nofill=layer_m4nofill,
        layer_m5nofill=layer_m5nofill,
        layer_tm1nofill=layer_tm1nofill,
        layer_tm2nofill=layer_tm2nofill,
        layer_text=layer_text,
        layer_metal5pin=layer_metal5pin,
        layer_topmetal1pin=layer_topmetal1pin,
        layer_metal5label = layer_metal5label,
        layer_topmetal1label = layer_topmetal1label,
        model=model
    )
    c.info = cap.info
    c.add_ref(cap)

    # add pwell block
    size = c.bbox_np()
    size = size[1] - size[0]
    pwellblock_enc = 2.4
    pwell = gf.components.rectangle(
        size=(size[0] + 2*pwellblock_enc, size[1]+2*pwellblock_enc),
        layer=layer_pwellblock,
        centered=True,)
    ccenter = (c.x, c.y)
    pwell_ref = c.add_ref(pwell)
    pwell_ref.x = ccenter[0]
    pwell_ref.y = ccenter[1]

    # add p guard ring
    pguardring_seq = 0.6
    pguardring_width = 2.0
    c.add_ref(guard_ring(
        width = pguardring_width,
        guardRingSpacing = pguardring_seq,
        guardRingType = 'psub',
        bbox = c.bbox_np(),
        path = None,
        layer_activ = layer_activ,
        layer_cont = layer_cont,
        layer_metal1 = layer_metal1,
        layer_psd = layer_psd,))
    
    logic_layers = [
        layer_activnoqrc,
        layer_metal1noqrc,
        layer_metal2noqrc,
        layer_metal3noqrc,
        layer_metal4noqrc,
        layer_metal5noqrc,
        layer_topmetal1noqrc
    ]

    size = c.bbox_np()
    size = size[1] - size[0]
    ccenter = (c.x, c.y)
    for layer_spec in logic_layers:
        ll = gf.components.rectangle(
            size=(size[0], size[1]),
            layer=layer_spec,
            centered=True,)
        ref = c.add_ref(ll)
        ref.x = ccenter[0]
        ref.y = ccenter[1]

    gr_drc = {
        'active_min_enclose_pp': 0.14,
    }

    # add TIE LOW pin
    tie_low = gf.components.rectangle(
        size=(size[0] - 2*gr_drc['active_min_enclose_pp'], pguardring_width),
        layer=layer_metal1pin,
        centered=True,)
    xmin, ymin = c.xmin, c.ymin
    tie_low_ref = c.add_ref(tie_low)
    tie_low_ref.xmin = xmin + gr_drc['active_min_enclose_pp']
    tie_low_ref.ymin = ymin + gr_drc['active_min_enclose_pp']
    
    tie = c.add_port(
        name="TIE_LOW",
        center=(tie_low_ref.x, tie_low_ref.y),
        width=pguardring_width,
        orientation=0,
        layer=layer_metal1,
        port_type="electrical",
    )
    c.add_label(
        text="TIE_LOW",
        position=(tie.x, tie.y),
        layer=layer_metal1label
    )
    c.add_label(
        text="TIE_LOW",
        position=(tie.x, tie.y),
        layer=layer_text
    )

    c.info['model'] = model

    return c



In [99]:
c = Component()
cap = rfcmim()
c.add_ref(cap)

Unnamed_254: ports ['TIE_LOW'], KCell(name=Unnamed_255, ports=['TIE_LOW'], pins=[], instances=['Unnamed_256_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S12p71_ca79e953_0_0', 'Unnamed_257_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__7b8b281a_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__1fbc386d_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__cd11e7fd_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__c032e132_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__63c278d1_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__e3648634_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S18p2__0ec3c713_0_0', 'rectangle_gdsfactorypcomponentspshapesprectangle_S17p91_3ba1c72e_0_-7960'], locked=False, kcl=DEFAULT)

In [100]:
c.write_gds(GDS_PATH / "rfcmim_example.gds")

PosixPath('/mnt/c/Users/dasdi/Documents/GDSFactory/diogo/cells/design_data/gds/rfcmim_example.gds')